# backtesting

A quick test of the portfolio methodology via backtesting, with some helpful functions. 

## Imports

In [29]:
import pandas as pd
import quandl as ql
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import warnings

# Make Chart dimensions waaaay bigger, some ptplot option
%matplotlib inline 
# Less busy convex solver logs
solvers.options['show_progress'] = False

## Data Retrieval

Retrieving high fidelity data from (Kaggle.)[https://www.kaggle.com/jessevent/all-crypto-currencies/data] Would prefer Quandl, but costs money.

In [6]:
df = pd.read_csv('data/crypto.csv')

In [63]:
symbols = df.symbol.unique()
slugs = df.slug.unique()

In [67]:
df.loc[df.symbol == 'STEEM'].tail(100)

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
18093,steem,STEEM,Steem,2017-09-25,27,1.070000,1.17000,1.040000,1.110000,778463,260006000,0.5385,0.13
18094,steem,STEEM,Steem,2017-09-26,27,1.110000,1.12000,1.070000,1.090000,850509,268720000,0.4000,0.05
18095,steem,STEEM,Steem,2017-09-27,27,1.080000,1.26000,1.080000,1.220000,2507270,263203000,0.7778,0.18
18096,steem,STEEM,Steem,2017-09-28,27,1.220000,1.49000,1.200000,1.430000,6612830,295714000,0.7931,0.29
18097,steem,STEEM,Steem,2017-09-29,27,1.440000,1.64000,1.260000,1.640000,9012310,349458000,1.0000,0.38
18098,steem,STEEM,Steem,2017-09-30,27,1.640000,1.68000,1.430000,1.470000,5720930,397770000,0.1600,0.25
18099,steem,STEEM,Steem,2017-10-01,27,1.460000,1.52000,1.410000,1.410000,3063030,354133000,0.0000,0.11
18100,steem,STEEM,Steem,2017-10-02,27,1.390000,1.43000,1.290000,1.310000,2974760,337053000,0.1429,0.14
18101,steem,STEEM,Steem,2017-10-03,27,1.320000,1.33000,1.170000,1.230000,1528740,318943000,0.3750,0.16
18102,steem,STEEM,Steem,2017-10-04,27,1.220000,1.23000,1.130000,1.180000,1306770,295222000,0.5000,0.10


In [69]:
required_symbols = ['BTC','ETH','DASH','XMR','QTUM','WAVES','LSK','OMG','STEEM','REP', 'GNT','SC','DGD','FCT','MAID','PAY','SNT','BAT','ZRX','BNT','CVC','STORJ','SINGLS','ANT','RLC','TRST','DNT']

BLX: 

* Bitcoin (BTC)	Value	15.00 %	7.04 %
* Ethereum (ETH)	Platform	14.17 %	14.49 %
* Dash (DASH)	Anonymity	12.89 %	8.36 %
* Monero (XMR)	Anonymity	12.68 %	8.48 %
* Qtum (QTUM)	Platform	6.90 %	10.82 %
* Waves (WAVES)	Platform	6.50 %	3.22 %
* Lisk (LSK)	Platform	6.21 %	9.20 %
* OmiseGO (OMG)	Decentralized Exchange and Payments Platform	5.62 %	6.55 %
* Steem (STEEM)	Media	2.41 %	3.54 %
* Augur (REP)	Prediction	2.02 %	2.66 %
* Golem (GNT)	Decentralized Computing	1.75 %	2.04 %
* Siacoin (SC)	Storage	1.60 %	5.07 %
* DigixDAO (DGD)	Value Gold	1.47 %	1.22 %
* Factom (FCT)	Audit	1.33 %	1.52 %
* MaidSafeCoin (MAID)	Storage	1.27 %	1.19 %
* TenX (PAY)	Debit Card	1.27 %	1.04 %
* Status (SNT)	Smartphone Ethereum Gateway	1.20 %	3.56 %
* Basic Attention Token (BAT)	Advertisement	1.18 %	2.10 %
* 0x (ZRX)	Decentralized Exchange Protocol	0.86 %	2.90 %
* Bancor (BNT)	Financial Derivative	0.78 %	1.09 %
* Civic (CVC)	Identity Management	0.72 %	0.90 %
* Storj (STORJ)	Storage	0.54 %	0.72 %
* SingularDTV (SNGLS)	Media / TV	0.48 %	0.55 %
* Aragon (ANT)	Enterprise Management	0.43 %	0.54 %
* iExec (RLC)	Decentralized Computing	0.33 %	0.49 %
* WeTrust (TRST)	Lending	0.24 %	0.25 %
* District0x (DNT)	Decentralized infrastructure	0.15 %	0.46 %

In [65]:
print(slugs)
print(symbols)

['bitcoin' 'ripple' 'ethereum' ..., 'aidos-kuneen' 'casinocoin'
 'bitcoin-unlimited']
['BTC' 'XRP' 'ETH' ..., 'ADK' 'CSC' 'BTU']


In [66]:
[print(i) for i in slugs]
[print(i) for i in symbols]

bitcoin
ripple
ethereum
bitcoin-cash
cardano
litecoin
stellar
nem
iota
dash
monero
neo
tron
eos
bitcoin-gold
qtum
raiblocks
ethereum-classic
bitconnect
icon
lisk
bitshares
verge
omisego
status
ardor
steem
zcash
populous
stratis
tether
waves
bytecoin-bcn
hshare
dogecoin
komodo
siacoin
golem-network-tokens
binance-coin
augur
vechain
veritaseum
ark
kucoin-shares
decred
digibyte
salt
pivx
dragonchain
nxt
factom
aion
monacoin
byteball
basic-attention-token
maidsafecoin
request-network
zcoin
enigma-project
power-ledger
bitcoindark
civic
0x
reddcoin
tenx
experience-points
electroneum
kyber-network
wax
syscoin
santiment
funfair
ethos
bytom
rchain
digixdao
qash
kin
xtrabytes
aeternity
gas
vertcoin
gnosis-gno
gamecredits
storj
iconomi
quantstamp
dent
substratum
aelf
gxshares
skycoin
ubiq
raiden-network-token
walton
centra
decentraland
bancor
storm
bitbay
blocknet
chainlink
edgeless
smartcash
poet
red-pulse
time-new-bank
iot-chain
triggers
revain
nav-coin
cryptonex
monaco
einsteinium
bridgecoin
r

beavercoin
blackstar
bitquark
dreamcoin
jewels
mindcoin
asiadigicoin
cryptoworldx-token
litecred
morningstar-payments
billarycoin
vip-tokens
pulse
warp
cashcoin
quebecoin
uro
ego
orlycoin
dappster
dpay
crtcoin
plncoin
zonecoin
mtmgaming
bios-crypto
playercoin
impulsecoin
zayedcoin
bowscoin
jobscoin
ponzicoin
tagrcoin
genstake
high-voltage
wild-beast-block
arbit
letitride
posex
agrolifecoin
speedcash
xonecoin
osmiumcoin
gbcgoldcoin
antilitecoin
kingn-coin
ibank
sydpak
cabbage
cryptoescudo
futurexe
p7coin
ccminer
corethum
selfiecoin
biobar
nodecoin
qibuck-asset
socialcoin-socc
ulatech
save-and-gain
concoin
lex4all
digital-credits
pizzacoin
caliphcoin
applecoin-apw
atmcoin
genaro-network
nebulas-token
bibox-token
smartmesh
entcash
bitcoin-diamond
energo
dimcoin
medishares
naga
html-coin
prochain
segwit2x
qlink
comsa-eth
hyper-pay
micromoney
olympus-labs
sophiatx
dew
tezos
bitclave
bitcoinx
clubcoin
homeblockcoin
comsa-xem
qbao
super-bitcoin
fargocoin
embercoin
lltoken
poly-ai
cappasity
in

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [86]:
coins = []

[print(name) if (len(df.loc[df.symbol == name].tail(365))>0) else print('FUCK',name) for name in required_symbols]

BTC
ETH
DASH
XMR
QTUM
WAVES
LSK
OMG
STEEM
REP
GNT
SC
DGD
FCT
MAID
PAY
SNT
BAT
ZRX
BNT
CVC
STORJ
FUCK SINGLS
ANT
RLC
TRST
DNT


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [75]:
df.loc[df.symbol == 'QTUM'].tail(100)

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
12348,qtum,QTUM,Qtum,2017-09-25,16,8.03,9.60,8.03,9.36,82385100,473818000,0.8471,1.57
12349,qtum,QTUM,Qtum,2017-09-26,16,9.38,10.30,9.17,9.34,104988000,553598000,0.1504,1.13
12350,qtum,QTUM,Qtum,2017-09-27,16,9.37,9.90,9.35,9.90,59847700,552811000,1.0000,0.55
12351,qtum,QTUM,Qtum,2017-09-28,16,9.95,10.10,9.06,9.93,95333300,586983000,0.8365,1.04
12352,qtum,QTUM,Qtum,2017-09-29,16,9.96,10.00,9.15,9.77,79526300,587845000,0.7294,0.85
12353,qtum,QTUM,Qtum,2017-09-30,16,9.77,11.65,9.75,11.59,141320000,576371000,0.9684,1.90
12354,qtum,QTUM,Qtum,2017-10-01,16,11.59,12.61,11.58,12.34,162762000,683566000,0.7379,1.03
12355,qtum,QTUM,Qtum,2017-10-02,16,12.36,12.38,11.43,11.88,69789000,729062000,0.4737,0.95
12356,qtum,QTUM,Qtum,2017-10-03,16,11.91,12.12,10.78,12.12,95756400,702872000,1.0000,1.34
12357,qtum,QTUM,Qtum,2017-10-04,16,12.16,12.28,11.51,11.63,77387000,717223000,0.1558,0.77


In [43]:
daily = pd.date_range('1/3/2016', end='1/3/2018', freq='D')
weekly = pd.date_range('1/3/2016', end='1/3/2018', freq='W')
print(daily)
print(weekly)

DatetimeIndex(['2016-01-03', '2016-01-04', '2016-01-05', '2016-01-06',
               '2016-01-07', '2016-01-08', '2016-01-09', '2016-01-10',
               '2016-01-11', '2016-01-12',
               ...
               '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28',
               '2017-12-29', '2017-12-30', '2017-12-31', '2018-01-01',
               '2018-01-02', '2018-01-03'],
              dtype='datetime64[ns]', length=732, freq='D')
DatetimeIndex(['2016-01-03', '2016-01-10', '2016-01-17', '2016-01-24',
               '2016-01-31', '2016-02-07', '2016-02-14', '2016-02-21',
               '2016-02-28', '2016-03-06',
               ...
               '2017-10-29', '2017-11-05', '2017-11-12', '2017-11-19',
               '2017-11-26', '2017-12-03', '2017-12-10', '2017-12-17',
               '2017-12-24', '2017-12-31'],
              dtype='datetime64[ns]', length=105, freq='W-SUN')


In [47]:
opt.matrix(np.cov(eth['open'], btc['open']))

<2x2 matrix, tc='d'>

In [33]:
np.asmatrix(eth['open'])

matrix([[  2.83000000e+00,   2.79000000e+00,   7.06136000e-01,
           7.13989000e-01,   7.08087000e-01,   1.06000000e+00,
           1.22000000e+00,   1.81000000e+00,   1.80000000e+00,
           1.68000000e+00,   1.58000000e+00,   1.22000000e+00,
           1.17000000e+00,   1.25000000e+00,   1.48000000e+00,
           1.40000000e+00,   1.38000000e+00,   1.35000000e+00,
           1.23000000e+00,   1.13000000e+00,   1.17000000e+00,
           1.15000000e+00,   1.19000000e+00,   1.18000000e+00,
           1.32000000e+00,   1.35000000e+00,   1.35000000e+00,
           1.30000000e+00,   1.26000000e+00,   1.28000000e+00,
           1.34000000e+00,   1.30000000e+00,   1.25000000e+00,
           1.24000000e+00,   1.21000000e+00,   1.16000000e+00,
           9.87228000e-01,   1.04000000e+00,   9.40566000e-01,
           8.75189000e-01,   9.41977000e-01,   9.06865000e-01,
           8.74574000e-01,   8.49603000e-01,   8.81191000e-01,
           9.41496000e-01,   9.19337000e-01,   9.035600

In [36]:
## NUMBER OF ASSETS
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

return_vec = np.random.randn(n_assets, n_obs)

In [38]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

random_portfolio(return_vec)

(matrix([[ 0.00948996]]), matrix([[ 0.51000164]]))

### Todo:
* Pick the crypto to put in the portfolio (maybe top 20 by market cap in the data?)
* Generate a weight vector every week in the data, simulating real life (trained on the past year to date? All historical to current? Last week of data only?) 
* Given this weight vector, simulate a portfolio of that position for the week, and calculate expected performance.